In [1]:
!nvidia-smi

Sat Sep 28 08:51:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM3-32GB           Off |   00000000:05:00.0 Off |                    0 |
| N/A   28C    P0             45W /  350W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install pandas openpyxl sentence-transformers fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 73.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 62.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 KB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 KB 30.2 MB/s eta 0:00:00


In [10]:
!pwd

/home/user1


In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
from tqdm import tqdm
import pickle as pkl
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
faq = pd.read_excel("01_База_знаний.xlsx")
faq.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [14]:
faq.shape

(326, 5)

In [16]:
faq_real = pd.read_excel("02_Реальные_кейсы.xlsx")
faq_real.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [19]:
faq_real = faq_real[['Вопрос пользователя', 'Ответ сотрудника']].rename(columns={'Вопрос пользователя': 'Вопрос из БЗ', 'Ответ сотрудника' :'Ответ из БЗ'})

In [25]:
new_data = pd.concat([faq, faq_real])

In [27]:
new_data.shape

(1127, 5)

In [26]:
new_data.head(1)

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео


In [30]:
new_data.to_csv('data.csv')

In [29]:
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
top_k_hits = 1
model = SentenceTransformer(MODEL_NAME)
torch.save(model, 'modelbert.pt')

def get_data():
    #Распаковка данных
    return list(new_data['Вопрос из БЗ'])


def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} time: {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timing_decorator
def embedding():
    question = get_data()
    embeddings = []
    for q in tqdm(question, desc='Embedding'):
        embeddings.append(model.encode(q))
    with open('model.pkl', 'wb') as f:
        pkl.dump(embeddings, f)
    return embeddings

embeddings = embedding()


Embedding: 100%|██████████| 1127/1127 [00:06<00:00, 176.37it/s]

embedding time: 6.40 seconds


In [31]:
question = get_data()
sbert_embeddings = [torch.tensor(embedding) for embedding in embeddings]
def question_response(inp_question):
  question_embedding = model.encode(inp_question)
  correct_hits = util.semantic_search(question_embedding, sbert_embeddings, top_k=1)[0]
  return question[correct_hits[0]['corpus_id']]

In [32]:
correct = 0
for q in question:
  if question_response(q) == q:
    correct += 1

In [33]:
correct / len(embeddings)

0.9982253771073647